In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import pandas as pd
import numpy as np
import plotly.express as px

import utils as u

ModuleNotFoundError: No module named 'utils'

In [2]:
# %pip install currencyconverter
# %pip install pendulum
# %pip install "dask[complete]" 
# %pip install tqdm
from currency_converter import CurrencyConverter as cc, RateNotFoundError
from datetime import date, timedelta
import pendulum as pndlm
from tqdm import tqdm
import dask.dataframe as dd

# import graph_objects from plotly package
import plotly.graph_objects as go
 
# import make_subplots function from plotly.subplots
# to make grid of plots
from plotly.subplots import make_subplots

In [60]:
# Read file
def read_file(filename, sep='\n'):
    with open(filename, 'r') as f:
        lines = f.readlines()
    parsed_lines = [line.replace(sep,'') for line in lines]
    return parsed_lines

In [61]:
jrnl = read_file(jrnl_path)

In [193]:
with open('/home/deuces/Drive/Dev/failed_files.txt', 'w') as outfile:
  outfile.write('\n'.join(str(file) for file in act_failed_files))

In [ ]:
def convert(from_c,to_c,on_date):
    try:
        val = cc().convert(1, from_c, to_c, date=on_date)
    except RateNotFoundError as rnfe:
        val = None
    return val

In [ ]:
# exchg_dask['inr_to_usd'] = exchg_dask.apply(lambda row: convert('INR', 'USD',row["exch_date"]),axis=1, meta='float').compute()
exchg_dask['inr_to_eur'] = exchg_dask.apply(lambda row: convert('INR', 'EUR',row["exch_date"]),axis=1, meta='float').compute()
# exchg_dask['eur_to_usd'] = exchg_dask.apply(lambda row: convert('EUR', 'USD',row["exch_date"]),axis=1, meta='float').compute()

In [ ]:
exchg_df = exchg_dask.compute()

In [ ]:
def multi_plot(df, x, y1, y2, y3, yname1, yname2, yname3):
    # use specs parameter in make_subplots function
    # to create secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # plot a scatter chart by specifying the x and y values
    # Use add_trace function to specify secondary_y axes.
    fig.add_trace(
        go.Scatter(
            x=df[x].tolist(), y=df[y1].tolist(),
            name=yname1, yaxis="y1"
        )
    )
    
    # Use add_trace function and specify secondary_y axes = True.
    fig.add_trace(
        go.Scatter(
            x=df[x].tolist(), y=df[y2].tolist(),
            name=yname2, yaxis="y2"
        )
    )

    fig.add_trace(
        go.Scatter(
            x=df[x].tolist(), y=df[y3].tolist(),
            name=yname3, yaxis="y3"
        )
    )
    # Naming y-axes
    # fig.update_yaxes(title_text='INR to USD', secondary_y=False)
    # fig.update_yaxes(title_text='EUR to USD', secondary_y=True)
    return fig

In [ ]:
fig = multi_plot(exchg_df, x='exch_date', y1='inr_to_usd',y2='eur_to_usd', y3='inr_to_eur', yname1='INR to USD',yname2='EUR to USD', yname3='INR to EUR')

# Adding title text to the figure
fig.update_layout(
    title_text="INR and EUR to USD",
    yaxis=dict(
        title="INR to USD",
        titlefont=dict(
            color="blue"
        ),
        tickfont=dict(
            color="blue"
        )
    ),
     
    # pass the y-axis 2 title, titlefont, color and
    # tickfont as a dictionary and store it an
    # variable yaxis 2
    yaxis2=dict(
        title="EUR to USD",
        titlefont=dict(
            color="orange"
        ),
        tickfont=dict(
            color="orange"
        ),
        anchor="free",  # specifying x - axis has to be the fixed
        overlaying="y",  # specifyinfg y - axis has to be separated
        side="right",  # specifying the side the axis should be present
        position=1  # specifying the position of the axis
    ),
   
    # pass the y-axis 3 title, titlefont, color and
    # tickfont as a dictionary and store it an
    # variable yaxis 3
    yaxis3=dict(
        title="INR to EUR",
        titlefont=dict(
            color="green"
        ),
        tickfont=dict(
            color="green"
        ),
        anchor="x",     # specifying x - axis has to be the fixed
        overlaying="y",  # specifyinfg y - axis has to be separated
        side="right"  # specifying the side the axis should be present
    ),
    height = 900, template ='simple_white', font_family="BlexMono Nerd Font",
)
# Naming x-axis
fig.update_xaxes(title_text="Date ->")

fig.show()

In [ ]:
url = "https://s3.console.aws.amazon.com/s3/object/ryft-public-sample-data?region=us-east-1&prefix=esRedditJson/esRedditJson1"
uri = "s3://ryft-public-sample-data/esRedditJson/esRedditJson1"

In [ ]:
df = pd.read_csv('rent_invoices.csv')

In [ ]:
import boto3
from urllib.parse import urlparse
import logging
log = logging.getLogger(__name__)

In [ ]:
# S3 Functionality
def translate_s3_urn(urn, to="auto", region="auto"):
    """
    Translates between URL <=> URI
    """
    valid_tos = ["auto", "uri", "url"]
    assert to in valid_tos, f"'to' must be one of {valid_tos}"

    url_netloc = "s3.console.aws.amazon.com"
    if to == "uri":
        assert url_netloc in urn, "Not a valid S3 URL"

    elif to == "url":
        assert urn.startswith("s3://"), "Not a valid S3 URI"
        
        url_form = "https://{url_netloc}/s3/object/{bucket}?region={region}&prefix={prefix}"
        parsed = urlparse(urn,scheme="s3")
        
        translation = url_form.format(
            url_netloc = url_netloc,
            bucket = parsed.netloc,
            region=region,
            
        )
        bucket = parsed.netloc

    else:
        log.info("Auto detecting URN type based on scheme")
        to = "url" if urn.startswith("s3://") else "uri"
        translate_s3_urn(urn, to=to, region=region)
    
    return to

In [ ]:
def get_aws_session():
    session = boto3.session.Session()
    yield session

session = get_aws_session()

In [ ]:
translate_s3_urn(uri)

In [ ]:
parsed = urlparse(uri,scheme="s3")

In [ ]:
parsed

Q1a: Create a function pm_fee that computes the Property Management fee collected. 
* prior to 2021-07-01, we charged a flat $250 fee
* on 2021-07-01 we moved to a percent-based fee
* If the rent ≤ $2000, we charge 8%.
* If the rent is over $2000, we charge 8% for the first $2000 and 6% for any amount over thatmm

In [ ]:
df.info()

In [ ]:
def compute_total_fee(date_field, rent_amt):
    fee = 250 if date_field < '2021-07-01' else None
    if fee is None:
        if rent_amt <= 2000:
            fee = rent_amt * 0.08
        else:
            rent_over = rent_amt - 2000
            fee = 2000 * 0.08 + (rent_over * 0.06)

    return fee
        

In [ ]:
compute_total_fee('2021-08-01', 3000)

Q1b: How much did we collect in total pm_fees in 2022?

In [ ]:
df["invoice_year"] = df["invoice_date"].apply(lambda x: x.split('-')[0])

In [ ]:
df["invoice_fee"] = df.apply(lambda x: compute_total_fee(x['invoice_date'], x['rent_amount']), axis=1)

In [ ]:
df.head()

In [ ]:
df.groupby("invoice_year")['invoice_fee'].sum()

In [ ]:
df.groupby('market')['invoice_fee'].sum().sort_values(ascending=False)

Q3: Compute the YoY revenue increase by market.
What’s our fastest revenue-growing market?

dd

In [ ]:
yoy_df = df.groupby(["market","invoice_year"]).sum().sort_values(by=["market","invoice_year"]).reset_index()

In [ ]:
summary_df = yoy_df[["market","invoice_year","invoice_fee"]].pivot(index='market', columns='invoice_year',values='invoice_fee')

In [ ]:
summary_df.head()

In [ ]:
summary_df.apply(lambda row: (row['2022'] - row['2021']) * 100 / row['2021'], axis=1).sort_values(ascending=False)

Plot the number of occupied properties (collecting rent) over time

In [ ]:
plot_df = df.groupby(["invoice_date","market"])['property_id'].nunique().reset_index()

In [ ]:
plot_df = plot_df.rename(columns={"property_id":"num_properties"})

In [ ]:

px.line(plot_df, x='invoice_date', y='num_properties', color='market').show()

In [8]:
import requests
import json
import pandas as pd

In [106]:
uri='https://jsonmock.hackerrank.com/api/food_outlets?page=1'

In [37]:
def call_restaurant_api(params: dict = {'city':'Seattle'}):
    # uri = f'https://jsonmock.hackerrank.com/api/food_outlets?city={city}'
    uri = 'https://jsonmock.hackerrank.com/api/food_outlets'
    response = requests.get(uri, params=params)

    assert response.status_code == 200, f"API Error: {response.status_code}"

    parsed_response = json.loads(response.text)
    return parsed_response

In [48]:
total_pages = call_restaurant_api({'city':'Miami'})['total_pages']
outlet_df = pd.DataFrame()
for page in range(1,total_pages+1):
    response = call_restaurant_api({'city':'Miami', 'page':page})
    api_df = pd.DataFrame(response['data'])
    outlet_df = pd.concat([outlet_df, api_df])

In [49]:
outlet_df

,city,name,estimated_cost,user_rating,id
0,Miami,The Drunken Botanist,150,"{'average_rating': 4.3, 'votes': 4145}",11
1,Miami,Kamonegi,130,"{'average_rating': 4.6, 'votes': 12868}",12
2,Miami,Plum By Bent Chair,180,"{'average_rating': 4.5, 'votes': 308}",13
3,Miami,AIR- An Ivory Region,10,"{'average_rating': 4, 'votes': 901}",14
4,Miami,Local,160,"{'average_rating': 4.3, 'votes': 5117}",15
5,Miami,Joule,160,"{'average_rating': 4.4, 'votes': 7171}",16
6,Miami,AMA Cafe,450,"{'average_rating': 4.4, 'votes': 2929}",17
7,Miami,The Junkyard Cafe,150,"{'average_rating': 4.4, 'votes': 4998}",18
8,Miami,Pirates of Grill,200,"{'average_rating': 4.8, 'votes': 4879}",19
9,Miami,38 Barracks,160,"{'average_rating': 4.6, 'votes': 2872}",110


In [50]:
outlet_df[['average_rating','votes']] = pd.json_normalize(outlet_df['user_rating'])

In [53]:
outlet_df

,city,name,estimated_cost,user_rating,id,average_rating,votes
0,Miami,The Drunken Botanist,150,"{'average_rating': 4.3, 'votes': 4145}",11,4.3,4145
1,Miami,Kamonegi,130,"{'average_rating': 4.6, 'votes': 12868}",12,4.6,12868
2,Miami,Plum By Bent Chair,180,"{'average_rating': 4.5, 'votes': 308}",13,4.5,308
3,Miami,AIR- An Ivory Region,10,"{'average_rating': 4, 'votes': 901}",14,4.0,901
4,Miami,Local,160,"{'average_rating': 4.3, 'votes': 5117}",15,4.3,5117
5,Miami,Joule,160,"{'average_rating': 4.4, 'votes': 7171}",16,4.4,7171
6,Miami,AMA Cafe,450,"{'average_rating': 4.4, 'votes': 2929}",17,4.4,2929
7,Miami,The Junkyard Cafe,150,"{'average_rating': 4.4, 'votes': 4998}",18,4.4,4998
8,Miami,Pirates of Grill,200,"{'average_rating': 4.8, 'votes': 4879}",19,4.8,4879
9,Miami,38 Barracks,160,"{'average_rating': 4.6, 'votes': 2872}",110,4.6,2872


In [54]:
within_cost_df = outlet_df.loc[lambda df: df.estimated_cost <= 120,:]
highest_rated_df = within_cost_df.loc[lambda df: df.average_rating == df.average_rating.max(),:]
highest_rated_df.loc[lambda df: df.estimated_cost == df.estimated_cost.min(),:]

,city,name,estimated_cost,user_rating,id,average_rating,votes
8,Miami,The Burger Club,60,"{'average_rating': 4.1, 'votes': 2072}",129,4.8,4879
